In [2]:
import pandas as pd
import matplotlib as plt

##### What will happen to you if you were in a car accident?

In [25]:
# import chardet

# with open('./data/accident.CSV', 'rb') as filedata:
#     result = chardet.detect(filedata.read(10000000))
# result

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}

In [28]:
df = pd.read_csv('./data/Person.CSV', encoding='Windows-1252')
df.head()

/tmp/ipykernel_5163/861866498.py:1: DtypeWarning: Columns (15,106,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/Person.CSV', encoding='Windows-1252')


,STATE,STATENAME,ST_CASE,VE_FORMS,VEH_NO,PER_NO,STR_VEH,COUNTY,DAY,DAYNAME,...,WORK_INJ,WORK_INJNAME,HISPANIC,HISPANICNAME,LOCATION,LOCATIONNAME,HELM_USE,HELM_USENAME,HELM_MIS,HELM_MISNAME
0,1,Alabama,10001,2,1,1,0,81,7,7,...,8,Not Applicable (not a fatality),0,Not A Fatality (not Applicable),0,Occupant of a Motor Vehicle,20,Not Applicable,7,None Used/Not Applicable
1,1,Alabama,10001,2,1,2,0,81,7,7,...,0,No,7,Non-Hispanic,0,Occupant of a Motor Vehicle,20,Not Applicable,7,None Used/Not Applicable
2,1,Alabama,10001,2,2,1,0,81,7,7,...,8,Not Applicable (not a fatality),0,Not A Fatality (not Applicable),0,Occupant of a Motor Vehicle,20,Not Applicable,7,None Used/Not Applicable
3,1,Alabama,10002,2,1,1,0,55,23,23,...,0,No,7,Non-Hispanic,0,Occupant of a Motor Vehicle,20,Not Applicable,7,None Used/Not Applicable
4,1,Alabama,10002,2,2,1,0,55,23,23,...,8,Not Applicable (not a fatality),0,Not A Fatality (not Applicable),0,Occupant of a Motor Vehicle,20,Not Applicable,7,None Used/Not Applicable
